In [ ]:
import IPython
from IPython.display import display, HTML

display(HTML("<h1>Northwind Logistics Dashboard</h1>"))

# Load the HTML file generated by the script
with open("3d_dashboard.html", "r") as f:
    html_code = f.read()

display(HTML(html_code))
# 3. Commentary
print("\n" + "="*50)
print("INSIGHTS:")
print("1. Use the mouse to rotate the cube above.")
print("2. Green bubbles represent successful deliveries.")
print("3. Red bubbles represent undelivered orders.")
print("4. The size of the bubble indicates the number of orders.")
print("="*50)